### Example connecting to Azure SQL database using Azure CLI credentials, sqlalchemy, and pandas

-----

### Import libraries and retrieve environment variables

In [ ]:
#
# import libraries
#
import os
import pandas as pd
import struct
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL
from azure.identity import AzureCliCredential
#
# retrieve and print environment variables
#
# load environment variables from .env file (which should be in .gitignore)
# this is a way to keep sensitive information out of the codebase
# the following code allows for .env file to be in same directory as script
# or you can specify the path relative to the notebook to the .env file
from os.path import join
from dotenv import load_dotenv
dotenv_path = join(os.getcwd(), '.env')
load_dotenv(dotenv_path)
# get environment variables
DRIVER = os.getenv("DRIVER")
RESOURCE = os.getenv("RESOURCE")
SERVER = os.getenv("SERVER")
DATABASE = os.getenv("DATABASE")
SQL_COPT_SS_ACCESS_TOKEN = 1256  # Connection option for access tokens, as defined in msodbcsql.h
# print environment variables
print("Environment variables loaded successfully.")
print(f"Driver: {DRIVER}", f"Resource: {RESOURCE}", f"Server: {SERVER}", f"Database: {DATABASE}", sep="\n")

### Connect to Azure SQL database using Azure CLI credentials.

In [ ]:
conn_str = (
    f'DRIVER={DRIVER};'
    f'SERVER={SERVER};'
    f'DATABASE={DATABASE};'
    f'Connection Timeout=30;'
    # f'Authentication=ActiveDirectoryServicePrincipal;'
    # f'TenantId={tenant_id};'
    # f'UID={client_id};'
    # f'PWD={client_secret};'
    # f'Token={token};'
    # f'Trusted_Connection=No;'
    # f'Trusted_Connection=No;'
    # f'MultipleActiveResultSets=true;'
    # f'TrustServerCertificate=true;'
    # f'Trusted_Connection=No;'
)

# params = urllib.parse.quote_plus(conn_str)

connection_string = 'mssql+pyodbc:///?odbc_connect=' + conn_str

engine = create_engine(connection_string)
#
# get token from Azure CLI
#
credential = AzureCliCredential()
#
# connect using token
#
@event.listens_for(engine, "do_connect")
def provide_token(dialect, conn_rec, cargs, cparams):
    # remove the "Trusted_Connection" parameter that SQLAlchemy adds
    cargs[0] = cargs[0].replace(";Trusted_Connection=Yes", "")

    # create token credential
    raw_token = credential.get_token(RESOURCE).token.encode("utf-16-le")
    token_struct = struct.pack(f"<I{len(raw_token)}s", len(raw_token), raw_token)

    # apply it to keyword arguments
    cparams["attrs_before"] = {SQL_COPT_SS_ACCESS_TOKEN: token_struct}

if engine:
    print("Connection established successfully.")
else:
    print("Connection could not be established.")

### Query database and close connection

In [ ]:
#
# execute query and load results into dataframe
#
df = pd.read_sql('SELECT TOP 1 * FROM Person.Person', engine)
#
# print dataframe shape
#
print(f'Dataframe Shape: {df.shape}')
#
# close connection
#
engine.dispose()